In [1]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyspark.sql import SparkSession

In [2]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("PySpark and Neural Network") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/07 19:06:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/07 19:06:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/07 19:06:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Load the dataset
dataset_path = "obesity_dataset.csv"
df = spark.read.csv(dataset_path, header = True, inferSchema = True)

## 1. Height as predictor of obesity and overweight

In [4]:
# Select columns
data = df[['Height', 'Obese/Overweight']]
data.head()

Row(Height=1.62, Obese/Overweight='No')

In [5]:
# Convert to pandas dataframe
pandas_df = data.select("Height", "Obese/Overweight").toPandas()
pandas_df.head()

,Height,Obese/Overweight
0,1.62,No
1,1.52,No
2,1.80,No
3,1.80,Yes
4,1.78,Yes


In [6]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df['Obese/Overweight'] = label_encoder.fit_transform(pandas_df['Obese/Overweight'])

In [7]:
# Define features (X) and target (y)
X = pandas_df[['Height']] 
y = pandas_df['Obese/Overweight']   

In [8]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [12]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7310 - loss: 0.6394 - val_accuracy: 0.7544 - val_loss: 0.6111
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.7266 - loss: 0.6093 - val_accuracy: 0.7544 - val_loss: 0.5928
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.6935 - loss: 0.6274 - val_accuracy: 0.7544 - val_loss: 0.5763
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.7493 - loss: 0.5696 - val_accuracy: 0.7544 - val_loss: 0.5649
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.7252 - loss: 0.5849 - val_accuracy: 0.7544 - val_loss: 0.5617
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - accuracy: 0.7368 - loss: 0.5703 - val_accuracy: 0.7544 - val_loss: 0.5595
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.7296 - loss: 0.5802 - val_accuracy: 0.7544 - val_loss: 0.5576
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.7310 - loss: 0.5798 - val_accura

In [13]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5572 
Neural Network Accuracy: 0.7352246046066284


## 2. Age as predictor of obesity/overweight

In [14]:
# Select columns
data2 = df[['Age', 'Obese/Overweight']]
data2.head()

Row(Age=21.0, Obese/Overweight='No')

In [15]:
# Convert to pandas dataframe
pandas_df2 = data2.select("Age", "Obese/Overweight").toPandas()
pandas_df2.head()

,Age,Obese/Overweight
0,21.0,No
1,21.0,No
2,23.0,No
3,27.0,Yes
4,22.0,Yes


In [16]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df2['Obese/Overweight'] = label_encoder.fit_transform(pandas_df2['Obese/Overweight'])

In [17]:
# Define features (X) and target (y)
X = pandas_df2[['Age']]
y = pandas_df2['Obese/Overweight']   

In [18]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4964 - loss: 0.6948 - val_accuracy: 0.7544 - val_loss: 0.6456
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - accuracy: 0.7055 - loss: 0.6428 - val_accuracy: 0.7544 - val_loss: 0.5906
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7412 - loss: 0.5857 - val_accuracy: 0.7544 - val_loss: 0.5461
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - accuracy: 0.7445 - loss: 0.5447 - val_accuracy: 0.7574 - val_loss: 0.5182
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.7570 - loss: 0.5158 - val_accuracy: 0.7544 - val_loss: 0.5028
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.7317 - loss: 0.5219 - val_accuracy: 0.7722 - val_loss: 0.4941
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.7403 - loss: 0.5118 - val_accuracy: 0.7456 - val_loss: 0.4930
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.7318 - loss: 0.5078 - val_accura

In [23]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7904 - loss: 0.4347 
Neural Network Accuracy: 0.758865237236023


## 3. Meals per day as predictor of obesity/overweight

In [24]:
# Select columns
data3 = df[['Meals_Per_Day', 'Obese/Overweight']]
data3.head()

Row(Meals_Per_Day=3.0, Obese/Overweight='No')

In [25]:
# Convert to pandas dataframe
pandas_df3 = data3.select("Meals_Per_Day", "Obese/Overweight").toPandas()
pandas_df3.head()

,Meals_Per_Day,Obese/Overweight
0,3.0,No
1,3.0,No
2,3.0,No
3,3.0,Yes
4,1.0,Yes


In [26]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df3['Obese/Overweight'] = label_encoder.fit_transform(pandas_df3['Obese/Overweight'])

In [27]:
# Define features (X) and target (y)
X = pandas_df3[['Meals_Per_Day']]
y = pandas_df3['Obese/Overweight']   

In [28]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [32]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7052 - loss: 0.6698 - val_accuracy: 0.7544 - val_loss: 0.6336
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.7432 - loss: 0.6281 - val_accuracy: 0.7544 - val_loss: 0.6126
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.7382 - loss: 0.6092 - val_accuracy: 0.7544 - val_loss: 0.5991
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.7317 - loss: 0.6041 - val_accuracy: 0.7544 - val_loss: 0.5840
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.7233 - loss: 0.5976 - val_accuracy: 0.7544 - val_loss: 0.5675
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.7326 - loss: 0.5742 - val_accuracy: 0.7544 - val_loss: 0.5553
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.7132 - loss: 0.5803 - val_accuracy: 0.7544 - val_loss: 0.5486
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - accuracy: 0.7286 - loss: 0.5634 - val_accura

In [33]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8060 - loss: 0.4882 
Neural Network Accuracy: 0.7848699688911438


## 4. Frequency of physical activity as predictor of obesity/overweight

In [34]:
# Select columns
data4 = df[['Frequency_Physical_Activity', 'Obese/Overweight']]
data4.head()

Row(Frequency_Physical_Activity=0.0, Obese/Overweight='No')

In [35]:
# Convert to pandas dataframe
pandas_df4 = data4.select("Frequency_Physical_Activity", "Obese/Overweight").toPandas()
pandas_df4.head()

,Frequency_Physical_Activity,Obese/Overweight
0,0.0,No
1,3.0,No
2,2.0,No
3,2.0,Yes
4,0.0,Yes


In [36]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df4['Obese/Overweight'] = label_encoder.fit_transform(pandas_df4['Obese/Overweight'])

In [37]:
# Define features (X) and target (y)
X = pandas_df4[['Frequency_Physical_Activity']]
y = pandas_df4['Obese/Overweight']   

In [38]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [42]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4905 - loss: 0.6966 - val_accuracy: 0.7544 - val_loss: 0.6547
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.7331 - loss: 0.6540 - val_accuracy: 0.7544 - val_loss: 0.6259
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - accuracy: 0.7199 - loss: 0.6342 - val_accuracy: 0.7544 - val_loss: 0.6001
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.7318 - loss: 0.6062 - val_accuracy: 0.7544 - val_loss: 0.5792
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7236 - loss: 0.5941 - val_accuracy: 0.7544 - val_loss: 0.5666
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.7202 - loss: 0.5885 - val_accuracy: 0.7544 - val_loss: 0.5561
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.7137 - loss: 0.5915 - val_accuracy: 0.7544 - val_loss: 0.5516
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.7338 - loss: 0.5651 - val_accura

In [43]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5351 
Neural Network Accuracy: 0.7352246046066284


## 5. Water intake as predictor of obesity/overweight

In [44]:
# Select columns
data5 = df[['Water_Intake', 'Obese/Overweight']]
data5.head()

Row(Water_Intake=2.0, Obese/Overweight='No')

In [45]:
# Convert to pandas dataframe
pandas_df5 = data5.select("Water_Intake", "Obese/Overweight").toPandas()
pandas_df5.head()

,Water_Intake,Obese/Overweight
0,2.0,No
1,3.0,No
2,2.0,No
3,2.0,Yes
4,2.0,Yes


In [46]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df5['Obese/Overweight'] = label_encoder.fit_transform(pandas_df5['Obese/Overweight'])

In [47]:
# Define features (X) and target (y)
X = pandas_df5[['Water_Intake']]
y = pandas_df5['Obese/Overweight']   

In [48]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [49]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [52]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5129 - loss: 0.6978 - val_accuracy: 0.7544 - val_loss: 0.6462
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.7310 - loss: 0.6378 - val_accuracy: 0.7544 - val_loss: 0.5811
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7189 - loss: 0.5971 - val_accuracy: 0.7544 - val_loss: 0.5501
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.7255 - loss: 0.5882 - val_accuracy: 0.7544 - val_loss: 0.5441
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - accuracy: 0.7527 - loss: 0.5526 - val_accuracy: 0.7544 - val_loss: 0.5420
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - accuracy: 0.7372 - loss: 0.5645 - val_accuracy: 0.7544 - val_loss: 0.5418
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.7154 - loss: 0.5888 - val_accuracy: 0.7544 - val_loss: 0.5398
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - accuracy: 0.7429 - loss: 0.5665 - val_accuracy

In [53]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.4984 
Neural Network Accuracy: 0.7352246046066284


## 6. Vegetable intake as predictor of obesity/overweight

In [54]:
# Select columns
data6 = df[['Vegetable_Intake', 'Obese/Overweight']]
data6.head()

Row(Vegetable_Intake=2.0, Obese/Overweight='No')

In [55]:
# Convert to pandas dataframe
pandas_df6 = data6.select("Vegetable_Intake", "Obese/Overweight").toPandas()
pandas_df6.head()

,Vegetable_Intake,Obese/Overweight
0,2.0,No
1,3.0,No
2,2.0,No
3,3.0,Yes
4,2.0,Yes


In [56]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df6['Obese/Overweight'] = label_encoder.fit_transform(pandas_df6['Obese/Overweight'])

In [57]:
# Define features (X) and target (y)
X = pandas_df6[['Vegetable_Intake']]
y = pandas_df6['Obese/Overweight']   

In [58]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [59]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [62]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3327 - loss: 0.7198 - val_accuracy: 0.5651 - val_loss: 0.6888
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.6463 - loss: 0.6814 - val_accuracy: 0.7544 - val_loss: 0.6579
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.7312 - loss: 0.6558 - val_accuracy: 0.7544 - val_loss: 0.6301
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.7186 - loss: 0.6350 - val_accuracy: 0.7544 - val_loss: 0.6043
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.7274 - loss: 0.6155 - val_accuracy: 0.7544 - val_loss: 0.5836
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7338 - loss: 0.5972 - val_accuracy: 0.7544 - val_loss: 0.5716
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.7292 - loss: 0.5946 - val_accuracy: 0.7544 - val_loss: 0.5690
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.7254 - loss: 0.5943 - val_accuracy

In [63]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5615 
Neural Network Accuracy: 0.7352246046066284


## 7. Technology use as predictor of obesity/overweight

In [64]:
# Select columns
data7 = df[['Technology_Use', 'Obese/Overweight']]
data7.head()

Row(Technology_Use=1.0, Obese/Overweight='No')

In [65]:
# Convert to pandas dataframe
pandas_df7 = data7.select("Technology_Use", "Obese/Overweight").toPandas()
pandas_df7.head()

,Technology_Use,Obese/Overweight
0,1.0,No
1,0.0,No
2,1.0,No
3,0.0,Yes
4,0.0,Yes


In [66]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df7['Obese/Overweight'] = label_encoder.fit_transform(pandas_df7['Obese/Overweight'])

In [67]:
# Define features (X) and target (y)
X = pandas_df7[['Technology_Use']]
y = pandas_df7['Obese/Overweight']   

In [68]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [69]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [70]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [72]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6542 - loss: 0.6709 - val_accuracy: 0.7544 - val_loss: 0.6349
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.7502 - loss: 0.6243 - val_accuracy: 0.7544 - val_loss: 0.6027
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 669us/step - accuracy: 0.7319 - loss: 0.6053 - val_accuracy: 0.7544 - val_loss: 0.5826
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.7409 - loss: 0.5860 - val_accuracy: 0.7544 - val_loss: 0.5675
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7189 - loss: 0.5909 - val_accuracy: 0.7544 - val_loss: 0.5613
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7235 - loss: 0.5829 - val_accuracy: 0.7544 - val_loss: 0.5579
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.7388 - loss: 0.5605 - val_accuracy: 0.7544 - val_loss: 0.5536
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.7433 - loss: 0.5496 - val_accura

In [73]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.7515 - loss: 0.5629
Neural Network Accuracy: 0.7352246046066284
